In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
df_main = pd.read_csv("tagi_art1.csv", delimiter = "|", header = None)

In [ ]:
#remove columns without title, text or url
df_main = df_main.dropna(subset=[0])
df_main = df_main.dropna(subset=[4])
df_main = df_main.dropna(subset=[5])

In [ ]:
#data cleaninig
df_main[3] = df_main[3].fillna("Agentur")
df_main[2] = df_main[2].str.replace("Publiziert: ", "")

In [ ]:
#setting dates straight
time_indy = df_main[2]
time_indy = time_indy.str.split(",",expand=True)

df_main[2] = time_indy[0]
df_main[8] = time_indy[1]

df_main[2] = pd.to_datetime(df_main[2], format="%d.%m.%Y")
df_main[9] = df_main[2].dt.year

df_main = df_main[df_main[9] != 2021]
df_main = df_main.drop([9], axis = 1)
df_main = df_main.reset_index(drop = True)

In [ ]:
# drop duplicates
df_main = df_main.drop_duplicates(keep = "last", ignore_index = True)

In [ ]:
#setting up cateogries
catyy = df_main[5].str.split("/",expand=True)

catyy[8] = catyy[3]
catyy[9] = catyy[8].str.len()
catyy[9] = catyy[9].astype(int)

catyy[10] = np.where(catyy[9] < 20, catyy[3],"")

catyy[10].unique()


In [ ]:
df_main[9] = catyy[10]

In [ ]:
#reorder columns
df_main = df_main[[0,1,2,8,3,4,5,9]]


In [ ]:
df_main

In [ ]:
#settings up categories (ressorts) from topics
df_main[10] = ""
df_main[11] = df_main[9]
df_main[12] = ""


with open("ressorts_cat.bin", "rb") as data:
    ressorts = pickle.load(data)

df_main[11].replace(ressorts, inplace=True)

In [ ]:
#delete unworkables
with open("to_delete.bin", "rb")as data:
    to_delete = pickle.load(data)
    
df_main = df_main[~df_main[9].isin(to_delete)]

In [ ]:
df_main

In [ ]:
df_main = df_main[[0,1,2,8,3,4,5,9, 7, 8, 10]]
df_main

In [ ]:
df_main[13] = "tagi"
df_main[4] = df_main[4].str.replace("http\S+", "",regex = True)
df_main[4] = df_main[4].str.replace("Bild", "")

In [ ]:
df_main.to_csv("tagi_artikel.csv", header = None, sep = "|", index = False)

In [ ]:
df_main